>SCIKIT-LEARN

> **Group 23**
>> **Team-mates**
>>> **Madhav Jivani - 202201285**

>>> **Tarun Kumar**

>>> **Rupesh Singh**

In [6]:
# for using dataframes
import pandas as pd
# for data transformation or encoding and metric calculations
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# for feature selection and models
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.datasets import load_iris
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

In [8]:
data = pd.read_csv("loan_status_data.csv")
data.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


**SEPARTE THE NUMERIC AND CATEGORICAL DATA**

In [15]:
x_num = data[["id","person_age","person_income","person_emp_length","loan_amnt","loan_int_rate","loan_percent_income","cb_person_cred_hist_length"]]

x_cat = data[["id","person_home_ownership","loan_intent","loan_grade","cb_person_default_on_file","loan_status"]]
x_num.head()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
0,0,37,35000,0.0,6000,11.49,0.17,14
1,1,22,56000,6.0,4000,13.35,0.07,2
2,2,29,28800,8.0,6000,8.90,0.21,10
3,3,30,70000,14.0,12000,11.11,0.17,5
4,4,22,60000,2.0,6000,6.92,0.10,3


**CAPPING THE OUTLIERS TO 10 AND 90 PERCENTILE W.R.T. TO THEIR VALUES**

In [21]:
import pandas as pd

def remove_outliers(df):
    # Copy the original DataFrame to avoid modifying it directly
    cleaned_x_num = df.copy()
    
    # Iterate through each column
    for column in cleaned_x_num.columns:
        q75 = cleaned_x_num[column].quantile(0.75)
        q25 = cleaned_x_num[column].quantile(0.25)
        
        # Interquartile Range
        iqr = q75 - q25  
        lb = q25 - 1.5 * iqr  # Lower bound
        ub = q75 + 1.5 * iqr  # Upper bound
        
        # Mask the outliers
        cleaned_x_num[column] = cleaned_x_num[column].mask(
            (cleaned_x_num[column] < lb) | (cleaned_x_num[column] > ub)
        )
    
    # drop rows with any NaN values (fully remove rows with outliers)
    cleaned_x_num.dropna(inplace=True)
    
    return cleaned_x_num

# Use the function
cleaned_x_num = remove_outliers(x_num)
cleaned_x_num.head()


,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
0,0,37.0,35000.0,0.0,6000.0,11.49,0.17,14.0
1,1,22.0,56000.0,6.0,4000.0,13.35,0.07,2.0
2,2,29.0,28800.0,8.0,6000.0,8.90,0.21,10.0
3,3,30.0,70000.0,14.0,12000.0,11.11,0.17,5.0
4,4,22.0,60000.0,2.0,6000.0,6.92,0.10,3.0


**ENCODING THE CATEGORICAL DATA**

In [24]:
onehot_encoder = OneHotEncoder(sparse_output=False) 
label_encoder = LabelEncoder()

# One-Hot Encoding for nominal columns
nominal_cols = ['person_home_ownership', 'loan_intent', 'cb_person_default_on_file', 'loan_status']
onehot_encoded = onehot_encoder.fit_transform(x_cat[nominal_cols])

# Create DataFrame for One-Hot Encoded variables
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(nominal_cols))

# Label Encoding for ordinal columns
ordinal_col = 'loan_grade'
label_encoded = label_encoder.fit_transform(x_cat[ordinal_col])

# Create DataFrame for Label Encoded variable
label_encoded_df = pd.DataFrame(label_encoded, columns=[ordinal_col])

# Combine One-Hot Encoded, Label Encoded DataFrames, and the original id column
final_encoded_df = pd.concat([x_cat[['id']], onehot_df, label_encoded_df], axis=1)